# Projet 15 :  Options Américaines par régression
Eleves : __Arthur MARON__ et __Thi Kim Anh TRAN__

# Ressources et libraries
On importe différentes bibliothèques pour effectuer des calculs, des tracés et des générateurs de nombres aléatoires.

In [ ]:
# Librairies de calcul
import numpy as np
from scipy import stats

#  Librairies de tracés
import matplotlib.pyplot as plt

# Générateurs de nombres aléatoires
from numpy.random import default_rng
rng = default_rng(1234)

On définit les paramètres de l'option américaine que l'on souhaite évaluer. Il suffira de changer le valeurs de ce dictionnaire pour étudier l'influence des paramètres sur le prix final. Définir une variable globale permettra de recuperer les paramètres de l'option dans différentes fonctions qui suivront.

In [ ]:
# Les paramètres pour notre option contienne au moins 250 jours de trading pour une échéance à 1 an.
params = {
        "size_path" : 250,
        "size_sample" : int(1e6),
        "x0" : 100,
        "r" : 0.1,
        "sigma" : 0.25,
        "K" : 100,
        "T" : 1}

# On utilise on deuxième set de paramètres pour tester nos fonctions, moins couteuse en temps de calcul.
param_debug = {
        "size_path" : 10,
        "size_sample" : int(1e3),
        "x0" : 100,
        "r" : 0.1,
        "sigma" : 0.25,
        "K" : 100,
        "T" : 1}

# Cas n°1 : un seul sous jacent pour l'évaluation de la première option de payoff : $ (K - X_{t}^1)_{+} $  
On va définir une série de fonctions qui seront utilisées dans notre pricer américan dans le cas où notre payoff utilise un unique sous jacent. C'est donc un cas '1D'.

## Le payoff de l'option à évaluer
On commence par définir la fonction de payoff de l'option à évaluer.

In [ ]:
def payoff_1(n, x): 
    N, _, _, r, _, K, T = params.values()
    return np.exp(-r*n*T/N) * np.maximum(K-x, 0)

## Les bases de projection
L'algorithme de Longstaff-Schwartz utilise une regression linéaire sur une base construire à partir du regresseur X. En changeant et utilisant différentes bases, on peut influencer la convergence du pricer dans le calcul du prix de l'option. On décide d'utiliser quatre bases différents : 
- Constante + 3 dimensions
- Constante + 3 dimensions + payoff
- Constante + 5 dimensions
- Constante + 5 dimensions + payoff

In [ ]:
def base_1(x): 
    return np.array([np.ones_like(x),  x, x**2, x**3])

def base_2(x): 
    K = params["K"]
    return np.array([np.ones_like(x), np.maximum(K-x,0), x, x**2, x**3])

def base_3(x): 
    return np.array([np.ones_like(x),  x, x**2, x**3, x**4, x**5])

def base_4(x): 
    K = params["K"]
    return np.array([np.ones_like(x), np.maximum(K-x,0), x, x**2, x**3, x**4, x**5])

## La regression linéaire 
On implémente les fonction de calcul pour la regression linéaire. La première fonction calcul les paramètres thetas du modèle linéaire, et la deuxième effectue la prédiction linéaire.

In [ ]:
def reg_ols(payoff, X, base = base_1):
    norm = (base(X) @ base(X).T)    
    try : 
        return np.linalg.inv(norm) @ (base(X) @ payoff)
    except:
        return np.linalg.pinv(norm) @ (base(X) @ payoff)
    
def predict_ols(X, theta, base = base_1): 
    return np.dot(theta, base(X))

## Simulation des trajectoires
On doit maintenant programmer un simulateur de trajectoires pour un mouvement brownien géometrique. On se place dans le cas où le sous-jacent a une tendance égale au taux sans risque - cad le cas risque neutre.

In [ ]:
def simu_paths(size_path:int, size_sample:int, x0:float, r:float, sigma:float, K:float, T:float, 
               rndm = rng, payoff_function = payoff_1): 
    
    # Simulation des trajectoires
    h = T/size_path
    accr = np.sqrt(h) * rndm.standard_normal(size=(size_path, size_sample))
    sample = np.zeros(shape=(size_path+1, size_sample))
    sample[0] = x0
    for n in range(1, size_path+1):
        sample[n] = sample[n-1] * np.exp((r - 0.5 * sigma**2)*h + sigma*accr[n-1])
        
    # Simulation des cashflows
    cashflow = np.empty_like(sample)
    for n in range(0, size_path+1):
        cashflow[n] = payoff_function(n,sample[n])
    
    return sample, cashflow

In [ ]:
# Test
"""_, _ = simu_paths(**param_debug, rndm = default_rng(1234), payoff_function = payoff_1)"""

## Execution de l'algorithme de Longstaff-Schwartz
On code maintenant l'algorithme de LS qui effecture une backpropagation par regression linéaire.

In [ ]:
def longstaff_schwartz(sample, cashflow, base = base_1) :
    # Récuperation de la taille de l'échantillon
    size_path, size_sample = sample.shape[0]-1, sample.shape[1]

    # Nombre de bases pour la projection ols
    m = base(sample[0]).shape[0]   

    # Initialisation des paramètres de la regression linéaire
    thetas = np.zeros((size_path, m))     

    # Initialisation des temps d'arrets et payoffs optimaux
    optimal_stop = size_path * np.ones(size_sample, dtype=int)
    optimal_payoff = cashflow[size_path].copy()
        
    # Algorithme récusif backward
    for n in reversed(range(1, size_path)):
        thetas[n] = reg_ols(optimal_payoff, sample[n], base)
        is_optimal_n = cashflow[n] >= predict_ols(sample[n], thetas[n], base)    
        optimal_stop[is_optimal_n] = n 
        optimal_payoff[is_optimal_n] = cashflow[n, is_optimal_n].copy()
    
    return optimal_payoff, thetas

In [ ]:
# Test
"""xx, zz = simu_paths(**param_debug)
_ = longstaff_schwartz(xx, zz, base = base_2)"""

On utilise un l'algorithme de Monte-Carlo pour trouver le prix final de l'option. En effet, l'algorthme de LS permet d'obtenir un échantillon de prix discounté au temps 0 pour chaque trajectoire simulée. On utilise cet échantillon pour trouver le prix par Monte Carlo.

In [ ]:
def monte_carlo(sample, proba = 0.95):
    mean = np.mean(sample)
    var = np.var(sample, ddof=1)
    alpha = 1 - proba 
    quantile = stats.norm.ppf(1 - alpha/2)  
    ci_size = quantile * np.sqrt(var / sample.size)
    result = { 'mean': mean, 'var': var, 
               'lower': mean - ci_size, 
               'upper': mean + ci_size }
    return result

Finalement, on implémente une fonction générale qui coordonne l'ensemble des fonctions précédantes pour simuler les trajectoires, lancer Lonstaff-Schwartz, et donner un prix par Monte-Carlo.

In [ ]:
def us_pricer(size_path:int, size_sample:int, x0:float, r:float, sigma:float, K:float, T:float, 
                    rndm = rng, payoff_function = payoff_1, base = base_1, plot:bool=False) :
       
    # Simulation des trajectoires et des cashflows
    X, Z = simu_paths(size_path, size_sample, x0, r, sigma, K, T, rndm, payoff_function)
       
    # Lancement de l'algorithme de LS
    opt_payoff, thetas = longstaff_schwartz(X, Z, base)
    
    # Récuperation du prix par Monte Carlo
    price = monte_carlo(opt_payoff)
    
    # Tracé des courbes
    if plot :
        x_tmp = np.linspace(int(0.8*K), int(1.2*K), 1000)
        fig, ax = plt.subplots()
        ax.plot(x_tmp, payoff_function(size_path, x_tmp))
        for n in np.linspace(1, size_path-1, 10, dtype=int) :
            ax.plot(x_tmp, np.maximum(payoff_function(n, x_tmp), predict_ols(x_tmp, thetas[n], base)), label=fr"t={n}")
            ax.set_title(f"Les fonctions valeurs approchées.\nBase : {base.__name__}\nPrix : {price['mean']:.3f}")
            ax.legend()
        ax.grid()

    # Determination du prix par Monte Carlo
    return price

In [ ]:
# Test
"""us_pricer(**param_debug, rndm = default_rng(1234) , payoff_function = payoff_1, base = base_2, plot = True)"""

## Evaluation de la première option de payoff : $ (K - X_{t}^1)_{+} $  

In [ ]:
bases = [base_1, base_2, base_3, base_4]
prices = {i.__name__ : None for i in bases}

In [ ]:
for base in bases :
    prices[f"{base.__name__}"] = us_pricer(**params, rndm = default_rng(1234), 
                                           payoff_function = payoff_1, base = base, plot=True)

In [ ]:
# Test
"""prices"""

In [ ]:
def compare_res(prices:dict) :     
    # Limites du graphique
    tmp = list(prices.values())[0]
    tmp = tmp["mean"] - tmp["lower"]
    low = min([itm["lower"] - tmp for itm in prices.values()])
    high = max([itm["upper"] + tmp for itm in prices.values()])
    plt.ylim([low, high] )
    
    # Affichage des prix et des incertitudes
    plt.title("Comparaison des prix du pricer US selon les bases utilisées")
    plt.bar(prices.keys(), [itm["mean"] for itm in prices.values()], color="grey")
    plt.errorbar(prices.keys(), [itm["mean"] for itm in prices.values()], 
                 yerr = [itm["mean"]-itm["lower"] for itm in prices.values()] ,
                 fmt = 'none', capsize = 15, ecolor = 'red', elinewidth = 2, capthick = 2)
    
    # Affichage du texte
    for i, price in enumerate(prices.values()) :
        p, e = price["mean"], price["mean"] - price["lower"]
        plt.text(i, p, f"{p:.3f}", ha="right", fontsize=16.5)
        plt.text(i, p+e, f"±{e:.3f}", ha="center", fontsize=15, color="red")
    
    plt.grid()
    plt.show()

In [ ]:
compare_res(prices)

__Commentaires :__ Il y a deux regroupements distincts dans les prix : les bases qui ont un payoff et celles qui n'en ont pas. Les bases avec un payoff ont tendance à donner des prix Monte Carlo plus élevés que celles sans. Il est important de noter que, selon la théorie de Longstaff-Schwartz, le prix obtenu par régression linéaire est généralement inférieur au prix réel.

## Etude des performances

In [ ]:
%timeit us_pricer(**param_debug, rndm = default_rng(1234), payoff_function = payoff_1, base = base_2)

__Commentaires:__ On observe que le pricer met un temps de l'ordre de la milliseconde pour N = 10, ce qui correspond au nombre de pas utilisé pour l'option beermudéenne en cours de deuxième semestre.

In [ ]:
%timeit us_pricer(**params, rndm = default_rng(1234), payoff_function = payoff_1, base = base_2)

__Commentaires:__ Quand on augmente le nombre de pas pour se rapprocher d'une option américaine (on a pris N = 250 afin d'avoir au moins un point par jour de trading pour une option à échéance 1 an), le temps d'execution augmente considéreblement et passe à l'ordre de grandeur de la minute.

# Cas n°2 : plusieurs sous jacents pour l'évaluation de la deuxième option de payoff : $ (K - \sqrt{X_{t}^1 X_{t}^2})_{+} $ 
On va redéfinir une série de fonctions qui seront utilisées dans notre pricer américan dans le cas où notre payoff utilise plusieurs sous jacents. C'est donc un cas multidimensionnel.

On définit les paramètres de l'option dont on va évaluer le prix. Cette fois il faut définir deux ......

In [ ]:
params2 = {"size_path":250, 
            "size_sample":int(1e6), 
            
            # Paramètres du sous jacent 1
            "x0_1":110,             
            "sigma_1":0.45, 
           
           # Paramètres du sous jacent 2
            "x0_2":90, 
            "sigma_2":0.10,

            "r":0.1,
            "K":100, 
            "T":1}

In [ ]:
params2_debug = {"size_path":10, 
                "size_sample":int(1e3), 

                # Paramètres du sous jacent 1
                "x0_1":110,             
                "sigma_1":0.45, 

               # Paramètres du sous jacent 2
                "x0_2":90, 
                "sigma_2":0.10,

                "r":0.1,
                "K":100, 
                "T":1}

On doit redéfinir quelques fonctions en 2D.

## Le payoff de l'option à évaluer

In [ ]:
def payoff_2(n, X): 
    N, _, _, _, _, _, r, K, T = params2.values()
    return np.exp(-r * n * T/N) * np.maximum(K - np.sqrt(X[0] * X[1]), 0)

## Les bases de projection

On décide que les bases de projections s'occupent de la décomposition des dimensions. Ainsi, on n'a pas à changer certaines fonctions, mais seulement les bases qui doivent être renseignée à la main de toute facon. Ici, on teste plusieurs approches : - 

In [ ]:
def base_5(X): 
    x, y = X[0], X[1]
    return np.array([np.ones_like(x),  x*y, (x*y)**2, (x*y)**3])

def base_6(X): 
    x, y, K = X[0], X[1], params2["K"]
    return np.array([np.ones_like(x), np.maximum(K - np.sqrt(x * y), 0), x*y, (x*y)**2, (x*y)**3 ])

def base_7(X): 
    x, y = X[0], X[1]
    return np.array([np.ones_like(x),  x, x**2, x**3, y, y**2, y**3])

def base_8(X): 
    x, y, K = X[0], X[1], params2["K"]
    return np.array([np.ones_like(x),  np.maximum(K - np.sqrt(x * y), 0), x, x**2, x**3, y, y**2, y**3])

def base_9(X): 
    x, y, K = X[0], X[1], params2["K"]
    return np.array([np.ones_like(x), np.maximum(K - np.sqrt(x * y), 0), (x*y)**0.5, (x*y), (x*y)**2 ])

## Simulation des trajectoires en 2D
La fonction de simulation doit ....
Si on voulait avoir plus de 2 trajectoires, il faudrait ..... matrice de corrélation.....

In [ ]:
def simu_paths_2d(size_path:int, size_sample:int, 
                x0_1:float, sigma_1:float, 
                x0_2:float, sigma_2:float,  
                r:float, K:float, T:float, correlation:float, 
                rndm = rng, payoff_function = payoff_2):
    
    # Simulation des trajectoires
    h = T / size_path
    dW = np.sqrt(h) * rndm.standard_normal((2, size_path, size_sample))
    dB = dW.copy()
    dB[1] = correlation * dW[0] + np.sqrt(1-correlation**2) * dW[1]
    sample = np.zeros((2, size_path+1, size_sample))
    sample[0][0], sample[1][0] = x0_1, x0_2
    
    for n in range(1, size_path+1):
        sample[0][n] = sample[0][n-1] * np.exp((r - 0.5 * sigma_1**2)*h + sigma_1*dB[0][n-1])
        sample[1][n] = sample[1][n-1] * np.exp((r - 0.5 * sigma_2**2)*h + sigma_2*dB[1][n-1])
    
    # Simulation des cashflows
    cashflow = np.empty((size_path+1, size_sample))
    for n in range(0, size_path+1):
        cashflow[n] = payoff_function(n, sample[:, n, :] ) # [0][n], sample[1][n])
    
    return sample, cashflow

In [ ]:
# Test
"""tmp, cs = simu_paths_2d(**params2_debug, correlation = 0.8, rndm = default_rng(1234))
tmp[:, 1, :]"""

On teste notre fonction pour le mouvement brownine gémoetrique à dimension 2.

In [ ]:
def plot_trajectoires(parametres, correlation = 0.5, **sample) :
    times = np.arange(parametres["size_path"]+1)*(parametres["T"] / parametres["size_path"])
    plotted = False
    
    # Cas où on a renseigné des trajectoires et cashflow. Cela évite de re-simuler et sera utile pour size_path grand.
    for item, obj in sample.items():
        if item == "trajectoire" :
            X = obj
        if item == "cashflow":
            cs = obj
        plotted = True
    
    # Cas où on veut les simuler pour les afficher.
    if not plotted :
        X, cs = simu_paths_2d(**params2_debug, correlation = correlation, rndm = default_rng(1234))
    
    plt.plot(times, X[0,:,0], color='C0', label='X1')
    plt.plot(times, X[1,:,0], color='C1', label='X2')
    plt.plot(times, cs[:,0], color='C2', label='cashflow')
    plt.title("Simulation 2D d'un MBG") ; plt.legend() ; plt.grid() ; plt.show()

In [ ]:
plot_trajectoires(params2_debug)

## Algortigme de Longstaff Schwartz
On doit changer la fonction qui execute l'algortihme pour qu'elle prenne en compte la dimension supplémentaire.

In [ ]:
def longstaff_schwartz_multi_d(sample, cashflow, base = base_5) :
    # Taille de l'échantillon
    size_path, size_sample =  sample[0,1:,:].shape
    
    # Nombre de bases pour la projection ols
    m = base(sample[:, 0, :]).shape[0]   
    
    # Initialisation des paramètres de la regression linéaire
    thetas = np.zeros((size_path, m))            
    
    # Initialisation des temps d'arrets et payoffs optimaux
    optimal_stop = size_path * np.ones(size_sample, dtype=int)
    optimal_payoff = cashflow[size_path].copy()
        
    # Algorithme récusif backward
    for n in reversed(range(1, size_path)):
        thetas[n] = reg_ols(optimal_payoff, sample[:, n, :], base)
        is_optimal_n = cashflow[n] >= predict_ols(sample[:, n, :], thetas[n], base)    
        optimal_stop[is_optimal_n] = n 
        optimal_payoff[is_optimal_n] = cashflow[n, is_optimal_n].copy()
        
    return optimal_payoff, thetas

In [ ]:
#Test
"""xx, zz = simu_paths_2d(**params2_debug, correlation = 0.75)
plot_trajectoires(params2_debug, 0.75, trajectoire = xx, cashflow = zz)
opt_p, the = longstaff_schwartz_multi_d(xx, zz)
opt_p[:10]"""

Maintenant, on réécrit également la fonction de pricer pour deux sous-jacents.

In [ ]:
def us_pricer_2d(size_path:int, size_sample:int, 
                x0_1:float, sigma_1:float, x0_2:float, sigma_2:float,  
                r:float, K:float, T:float, correlation:float=0.5,
                rndm = rng, payoff_function = payoff_2, base = base_6, plot:bool=False) :
       
    # Simulation des trajectoires et des cashflows     
    X, Z = simu_paths_2d(size_path, size_sample, 
                 x0_1, sigma_1, x0_2, sigma_2, r, K, T, correlation = correlation, 
                 rndm = rndm, payoff_function = payoff_function)
    
    # Lancement de l'algorithme de LS
    opt_payoff, thetas = longstaff_schwartz_multi_d(X, Z, base)
    
    # Determination du prix par Monte Carlo
    price = monte_carlo(opt_payoff)
    
    # Tracé des courbes 3D
    if plot :
        fig, ax = plt.figure(), plt.axes(projection='3d')
        M = 10 # Résolution de l'affichage 3D
        x1_tmp = np.linspace(90, 130, M)
        x2_tmp = np.linspace(75, 140, M)
        sample_tmp = np.array([x1_tmp, x2_tmp])#.reshape((2, 1, 1000))   
        
        def plot_3d(t, c) :    
            payoff_t = np.zeros((M, M))
            for i, x1 in enumerate(x1_tmp) :
                for j, x2 in enumerate(x2_tmp) :
                    payoff_t[i, j] = payoff_function(t, np.array([x1, x2]))
            ax.plot_wireframe(x1_tmp, x2_tmp, payoff_t, label = f"t={t}", color=f"C{c}", alpha=0.5)
            ax.legend()
            """ Autre version 3D
            #ax.contour3D(x1_tmp, x2_tmp, payoff_t, 50)
            c1 = ax.plot_surface(x1_tmp, x2_tmp, payoff_t, rstride=1, cstride=1, cmap='viridis', 
                                 edgecolor='none', linewidths=0.2, label = f"t={t}")
            c1._facecolors2d=c1._facecolors3d
            c1._edgecolors2d=c1._edgecolors3d
            """
        for c, n in enumerate(np.linspace(1, size_path-1, 3, dtype=int)) :
            plot_3d(n, c)
        ax.set_title(f"Les fonctions valeurs approchées.\nBase : {base.__name__}\nPrix : {price['mean']:.3f}")
        plt.show()

    return price

In [ ]:
# Test
"""us_pricer_2d(**params2_debug, correlation = 0.5, rndm = default_rng(1234), 
             payoff_function = payoff_2, base = base_6, plot = True)"""

## Evaluation de la seconde option de payoff : $ (K - \sqrt{X_{t}^1 X_{t}^2})_{+} $ 

Comme pour la première option, on va évaluer l'option en utilisant plusieurs bases de proejction OLS.

In [ ]:
bases_2 = [base_5, base_6, base_7, base_8, base_9]
prices_2 = {i.__name__ : None for i in bases_2}

In [ ]:
for base in bases_2 :
    prices_2[f"{base.__name__}"] = us_pricer_2d(**params2, correlation = 0.55, 
                                              rndm = default_rng(1234) , payoff_function = payoff_2, base = base, plot = True)

On compare les prix finaux calculés par Monte Carlo selon les différents bases utilisées.

In [ ]:
compare_res(prices_2)

__Commentaires:__ On observe une plus grande différence entre les prix calculés pour les différentes bases, et notamment une différence important lorsque l'on ajoute/retire le payoff dans la base de projection.

## Etude des performances

In [ ]:
%timeit us_pricer_2d(**params2, correlation = 0.5, rndm = default_rng(1234), payoff_function = payoff_2, base = base_6)

__Commentaires:__ On que l'ordre de grandeur de la minute est aussi présent dans le cas de la deuxième option. Ce temps d'execution relativement long par rapport à nos habitudes d'instantanéité n'est tout de même pas inhabituel dans le monde de la finance.

# Etude numérique sur le prix du delta
Le but de cette section est d'utiliser les fonctionnalité de calcul de gradien automatique afin de trouver le delta de l'option à évaluer. On a essayer plusieurs approches, mais celle qui semble la plus polyvalente necessite pytorch. En effet, jax n'est pas disponible sur windows, et autograd (ancienne version de jax) fonctionne mal avec la modification d'arrays.

In [ ]:
import torch
torch.autograd.set_detect_anomaly(True)

On initialise de nouveaux paramètres pour l'option 1 à évaluer et dont on voudra calculer le delta.

In [ ]:
params_grad = {
        "size_path" : 10, 
        "size_sample" : int(10e3),
        "r" : torch.tensor([0.1], requires_grad=True), 
        "sigma" : torch.tensor([0.25], requires_grad=True), 
        "K" : torch.tensor([100.0], requires_grad=True), 
        "T" : torch.tensor([1.0], requires_grad=True) }

On va désormais ré-écrire nos fonctions à l'aide du module pytorch, en excluant numpy.

In [ ]:
def payoff_1_torch(n, x): 
    N, _, r, _, K, T = params_grad.values()
    return torch.exp(-r*n*T/N) * torch.maximum(K-x, torch.tensor([0.0], requires_grad=True) )

In [ ]:
# Test
"""payoff_1_torch(0, 98.)"""

## Bases de projection

In [ ]:
def base_10(x): 
    return torch.stack((torch.ones_like(x),  x, x**2, x**3))

def base_11(x): 
    K = params_grad["K"]
    return torch.stack([torch.ones_like(x), torch.maximum(K-x, torch.tensor([0.0])), x, x**2, x**3])

def base_12(x): 
    return torch.stack([torch.ones_like(x),  x, x**2, x**3, x**4, x**5])

def base_13(x): 
    K = params_grad["K"]
    return torch.stack([torch.ones_like(x), torch.maximum(K-x, torch.tensor([0.0])), x, x**2, x**3, x**4, x**5])

In [ ]:
#Test
"""tmp = torch.tensor([98.2, 99.99])
base_11(tmp)"""

## Fonction de regression OLS

In [ ]:
def reg_ols_torch(payoff, X, base = base_10):
    norm = (base(X) @ torch.transpose(base(X), 0, 1) )     
    try : 
        return torch.linalg.inv(norm) @ (base(X) @ payoff)
    except:
        return torch.linalg.pinv(norm) @ (base(X) @ payoff)
    
def predict_ols_torch(X, theta, base = base_10): 
    return torch.matmul(theta, base(X))

## Simulation des trajectoires

In [ ]:
def simu_paths_torch(x0, rndm = rng, payoff_function = payoff_1_torch): 
    
    # Récuperation des paramètres pour l'option en grad
    size_path, size_sample, r, sigma, K, T = params_grad.values()
    h = T/size_path
    
    # Simulation des trajectoires individuellement
    tmp = rndm.standard_normal(size=(size_path, size_sample))
    accr = torch.sqrt(h) * torch.from_numpy(tmp)    
    
    samples = list(torch.full((1, size_sample), x0.item(), requires_grad=True) )    
    for n in range(1, size_path+1):
        tmp1, tmp2 = samples[-1].clone(), accr[n-1].clone()
        samples.append(tmp1 * torch.exp((r - 0.5 * sigma**2) * h + sigma * tmp2 ))     
    sample = torch.stack(samples)
    #print("sample", sample)
    
    # Simulation des cashflows individuellement
    cashflows = [payoff_function(0, sample[0].clone())]     
    for n in range(1, size_path+1):
        cashflows.append(payoff_function(n, sample[n].clone()))
    cashflow =  torch.stack(cashflows)
    #print("cashflows", cashflow)
    
    # Retour
    return sample, cashflow

In [ ]:
xx = torch.tensor([100.0], requires_grad = True)
_,_ = simu_paths_torch(xx, rndm = default_rng(1234), payoff_function = payoff_1_torch)

## Longstaff-Schwartz

In [ ]:
def longstaff_schwartz_torch(sample, cashflow, base = base_10) :
    # Récuperation de la taille de l'échantillon
    size_path, size_sample = sample.shape[0] - 1, sample.shape[1]
    
    # Nombre de bases pour la projection ols ===============================================
    m = base(sample[0]).shape[0]      
    
    # Initialisation des temps d'arrets et payoffs optimaux ==================================
    optimal_stop = size_path * torch.ones(size_sample, requires_grad=True)
    optimal_payoff = cashflow[size_path].clone()
        
    #Calcul des thetas =========================================================
    thetas = list(torch.full((1, m), 0.0, requires_grad=True) )
    for n in reversed(range(1, size_path)):
        thetas.append(reg_ols_torch(optimal_payoff, sample[n].clone(), base))
    theta = torch.stack(thetas)
    #print("theta : ", theta)
    
    # Algorithme récusif backward ========================================================
    for n in reversed(range(1, size_path)):
        sample_n, theta_n = sample[n].clone(), theta[n].clone()
        pred = predict_ols_torch(sample_n, theta_n, base)
        is_optimal_n = cashflow[n].clone() >= pred
        
        optimal_stop[is_optimal_n] = n 
        #print("optimal_stop", optimal_stop, '\n', '-' * 50)
        
        optimal_payoff[is_optimal_n] = cashflow[n, is_optimal_n].clone()
        #print("optimal_payoff", optimal_payoff, '\n', '_' * 50)
    
    opt_out = optimal_payoff.clone().detach().requires_grad_(True)
    #return opt_out, theta    
    return optimal_payoff, theta

In [ ]:
vv = torch.tensor([100.0], requires_grad = True)
xx, zz = simu_paths_torch(vv, rndm = default_rng(1234))
#print(xx)
p, t = longstaff_schwartz_torch(xx, zz, base = base_10)
p

## Le pricer d'option américaine
Cette fois on fait en sorte qu'il ne dépende que de x0 l'utilisation du gradien.

In [ ]:
def us_pricer_raw(x0, rndm = rng, payoff_function = payoff_1_torch, base = base_10) :
    
    # Récuperation des paramètres de l'option
    #size_path, size_sample, r, sigma, K, T = params_grad.values()
    
    # Simulation
    X, Z = simu_paths_torch(x0, rndm, payoff_function)
    print("X", X)
    
    # Lonstaff Schwartz
    opt_payoff, _ = longstaff_schwartz_torch(X, Z, base)
    
    # Return les payoff optimaux
    return opt_payoff

In [ ]:
#Test
vv = torch.tensor([100.0], requires_grad = True)
us_pricer_raw(vv, rndm = default_rng(1234), payoff_function = payoff_1_torch, base = base_10)

In [ ]:
def price_mc(x):
    out_sample = us_pricer_raw(x, default_rng(1234), payoff_1_torch, base_10)    
    return torch.mean(out_sample)

In [ ]:
# Test
price_mc(torch.tensor([100.]))

## Différentiation automatique

In [ ]:
# Condition initiale en float
x0 = 109.2

# Condition initiale en pytorch
x0_torch = torch.tensor([x0], requires_grad = True)
print("x0_torch.requires_grad :", x0_torch.requires_grad)

# Calcul du prix
price_grad = price_mc(x0_torch)
print("price_grad.requires_grad :", price_grad.requires_grad, "\n")

# Calcul du gradient du prix
price_grad.backward()

# Affichage du gradient par du prix par x0
print(x0_torch.grad)

In [ ]:
# gradien = torch.autograd.grad(price_grad, x0_torch, create_graph=True)

Donne "None", ce qui n'est pas bon.

In [ ]:
def get_grad(x:float = 100.) :
    x0_torch = torch.tensor([x0], requires_grad = True)
    price_grad = price_mc(x0_torch)
    price_grad.backward()
    return x0_torch.grad

In [ ]:
print(get_grad(90.))